# Import библиотек

In [ ]:
!pip install langchain
!pip install sentence-transformers rank_bm25 --quiet
!pip install unstructured
!pip install chromadb
!pip install pdfminer.six
!pip install python-docx
!pip install typing_extensions
!pip install huggingface-hub
!pip install sentence-transformers
!pip install peft
!pip install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import accelerate
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

import shutil
import os
from huggingface_hub.file_download import http_get
from langchain.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    PDFMinerLoader,
    TextLoader,
    UnstructuredEmailLoader,
    UnstructuredEPubLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
)
from langchain.docstore.document import Document
from chromadb.config import Settings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document


from timeit import default_timer as timer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
MODEL_NAME = "IlyaGusev/saiga2_7b_lora" #IlyaGusev/saiga_mistral_7b_lora
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()

def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()




tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
config = PeftConfig.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()
torch.cuda.empty_cache()

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

tokenizer_config.json:   0%|          | 0.00/278 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/67.2M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 3584,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.5,
  "top_k": 30,
  "top_p": 0.9
}



In [ ]:
embedder_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
directory = './dataset_word/word'
file_paths = [os.path.join(directory, f) for f in os.listdir(directory)]
directory = './dataset_word/pdf'
for f in os.listdir(directory):
    file_paths.append(os.path.join(directory, f))

LOADER_MAPPING = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".enex": (EverNoteLoader, {}),
    ".epub": (UnstructuredEPubLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PDFMinerLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
}

# Upload files
def process_text(text):
    lines = text.split("\n")
    lines = [line for line in lines if len(line.strip()) > 2]
    text = "\n".join(lines).strip()
    if len(text) < 10:
        return None
    return text

def load_single_document(file_path: str) -> Document:
    ext = "." + file_path.rsplit(".", 1)[-1]
    assert ext in LOADER_MAPPING
    loader_class, loader_args = LOADER_MAPPING[ext]
    loader = loader_class(file_path, **loader_args)
    return loader.load()[0]



# Create database
def build_index(file_paths, chunk_size, chunk_overlap):
    documents = [load_single_document(path) for path in file_paths]
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    documents = text_splitter.split_documents(documents)
    fixed_documents = []
    for doc in documents:
        doc.page_content = process_text(doc.page_content)
        if not doc.page_content:
            continue
        fixed_documents.append(doc)

    db = Chroma.from_documents(
        fixed_documents,
        embeddings,
        client_settings=Settings(
            anonymized_telemetry=False
        )
    )

    return db

def retrieve(history, db, k_documents):
    context = ""
    if db:
        last_user_message = history[-1]
        retriever = db.as_retriever(search_kwargs={"k": k_documents})
        docs = retriever.get_relevant_documents(last_user_message)
        retrieved_docs = "\n\n".join([doc.page_content for doc in docs])
    return retrieved_docs, docs


chunk_size = 350 #500 #1000
chunk_overlap = 100
k_documents = 2
embeddings = HuggingFaceEmbeddings(model_name=embedder_name)

In [ ]:
start = timer()
db = build_index(file_paths, chunk_size, chunk_overlap)
end = timer()
print(f'time for building database = {end - start} seconds')
print("count of chunks in database =", len(db.get()['documents']))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def get_answer(message: str, return_chunks=False):
    history = []
    history.append(message)
    retrieved_docs, docs = retrieve(history, db, k_documents)

    conversation = Conversation()
    new_message = f"Контекст: {retrieved_docs}\n\n Используя контекст, ответь на вопрос:\n {message}"

    conversation.add_user_message(new_message)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)

    output += f"\n Релевантный документ по вопросу: {docs[0].metadata['source'].split('/')[-1]}"

    return output

In [ ]:
%%capture
!pip install pytelegrambotapi
!pip install gigachain
!pip install gigachat
!pip install gigachain-cli
!pip install notifiers
!pip install telebot

In [ ]:
CREDS = 'NWExYjc1NzAtMzYwMi00ODdlLWJmMGMtZGU0NWU5YjUwZDk4OjVlNDE0OTkwLTBkOTItNDIzOC1hM2JhLWI1MDhkYjE5NDZiNw=='
TG_API_TOKEN = '6414200086:AAHAXaaSbGYhx1IfMpo7s4SBRSdUxLqaGyE'

In [ ]:
import os
import string
import pickle
import telebot
from telebot import types
from typing import Any, Coroutine, List
from langchain.docstore.base import Document
from langchain.chains import RetrievalQA
from langchain.schema import HumanMessage, SystemMessage
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models.gigachat import GigaChat

In [ ]:
# Авторизация в сервисе GigaChat
chat = GigaChat(credentials=CREDS, verify_ssl_certs=False)

bot = telebot.TeleBot(TG_API_TOKEN)

keyboard = types.InlineKeyboardMarkup()
keyboard.add(
    types.InlineKeyboardButton(text='👍', callback_data='button_pressed_Ok'),
    types.InlineKeyboardButton(text='👎', callback_data='button_pressed_error')
)

instrucrion = 'Отвечай кратко. Желательно в одном предложении'

In [ ]:
@bot.callback_query_handler(func=lambda call: True)
def callback_handler(call):
    if call.data == 'button_pressed_Ok':
        bot.answer_callback_query(call.id, 'Ok! Я запомню🌟')
    elif call.data == 'button_pressed_error':
        temp_message_id = bot.send_message(call.message.json['chat']['id'], 'Ok! Попробую еще раз. Ожидайте...').message_id
        bot.answer_callback_query(call.id, '😫😫😫')

        if 'reply_to_message' in call.message.json:
            text = call.message.json['reply_to_message']['text']
        else:
            text = call.message.json['text']
        messages = [
        SystemMessage(content=instrucrion),
        HumanMessage(content=text)
        ]
        res = chat(messages)
        messages.append(res)
        bot.send_message(call.message.json['chat']['id'], res.content)
        bot.delete_message(call.message.json['chat']['id'], temp_message_id)

In [ ]:
@bot.message_handler(func=lambda message: True)
def echo_message(message):

    if message.text == '/start':
        bot.send_message(message.chat.id, 'Привет! Напиши вопрос')
    else:
        temp_message_id = bot.send_message(message.chat.id, 'Ожидайте, запрос выполняется...').message_id
        bot.reply_to(message, get_answer(message.text), reply_markup=keyboard)
        bot.delete_message(message.chat.id, temp_message_id)

In [ ]:
bot.polling()